In [1]:
import sourmash
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product, combinations
from sourmash.tax import tax_utils
from sourmash.lca import lca_utils

In [2]:
# download gtdb-rs207 taxonomy
!curl -JLO https://farm.cse.ucdavis.edu/\~ctbrown/sourmash-db/gtdb-rs207/gtdb-rs207.taxonomy.csv
# download signature info for k31
!curl -JLO https://farm.cse.ucdavis.edu/\~ctbrown/sourmash-db/gtdb-rs207/gtdb-rs207.genomic.k31.describe.csv
# download representative genome info
!curl -JLO https://farm.cse.ucdavis.edu/\~ctbrown/sourmash-db/gtdb-rs207/gtdb-rs207.taxonomy.reps.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.3M  100 57.3M    0     0  23.0M      0  0:00:02  0:00:02 --:--:-- 23.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 65.4M  100 65.4M    0     0  24.8M      0  0:00:02  0:00:02 --:--:-- 24.9M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11.6M  100 11.6M    0     0  9332k      0  0:00:01  0:00:01 --:--:-- 9348k


In [3]:
taxdb = 'gtdb-rs207.taxonomy.csv'
# read taxonomy into useful sourmash format
tax_assign = tax_utils.MultiLineageDB.load([taxdb],
                                           keep_full_identifiers=False,
                                           keep_identifier_versions=False )

In [4]:
# read in signature info
siginfocsv = "gtdb-rs207.genomic.k31.describe.csv"
sigInf = pd.read_csv(siginfocsv)
sigInf['ident'] = sigInf['name'].str.split(' ', expand=True)[0]
sigInf.head()

,signature_file,md5,ksize,moltype,num,scaled,n_hashes,seed,with_abundance,name,filename,license,sum_hashes,ident
0,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,bce936040b05e58a53be0974cbdef152,31,DNA,0,1000,4575,42,1,GCF_000814905.1 Enterobacter sp. Bisph1 strain...,/dev/fd/63,CC0,4594,GCF_000814905.1
1,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,2c46c657d26265d4ccc87b66a343e44e,31,DNA,0,1000,2404,42,1,"GCA_007116955.1 Vibrio sp., ASM711695v1",/dev/fd/63,CC0,2436,GCA_007116955.1
2,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,eea831872487c41dfc2fb613f493ba35,31,DNA,0,1000,5828,42,1,GCF_017948435.1 Pseudomonas protegens strain=M...,/dev/fd/63,CC0,5847,GCF_017948435.1
3,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,2c125d78f91b38497285d44e1b1d5405,31,DNA,0,1000,3306,42,1,"GCA_017995835.1 Amaricoccus sp., ASM1799583v1",/dev/fd/63,CC0,3332,GCA_017995835.1
4,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,dfc179236087a1222dbe18a61483fa61,31,DNA,0,1000,6960,42,1,GCF_001981135.1 Burkholderia pseudomallei stra...,/dev/fd/63,CC0,6993,GCF_001981135.1


In [5]:
sigInf['lineage'] = sigInf['ident'].apply(lambda x: tax_utils.find_match_lineage(x, tax_assign))
sigInf['lineage_str'] = sigInf['lineage'].apply(lambda x: lca_utils.display_lineage(x))
sigInf[['superkingdom', "phylum", "class", "order", "family", "genus", 'species']] = sigInf['lineage_str'].str.split(';', expand=True)

sigInf.head()

,signature_file,md5,ksize,moltype,num,scaled,n_hashes,seed,with_abundance,name,...,ident,lineage,lineage_str,superkingdom,phylum,class,order,family,genus,species
0,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,bce936040b05e58a53be0974cbdef152,31,DNA,0,1000,4575,42,1,GCF_000814905.1 Enterobacter sp. Bisph1 strain...,...,GCF_000814905.1,"((superkingdom, d__Bacteria), (phylum, p__Prot...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Enterobacteriaceae,g__Kosakonia,s__Kosakonia sp000814905
1,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,2c46c657d26265d4ccc87b66a343e44e,31,DNA,0,1000,2404,42,1,"GCA_007116955.1 Vibrio sp., ASM711695v1",...,GCA_007116955.1,"((superkingdom, d__Bacteria), (phylum, p__Prot...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Enterobacterales,f__Vibrionaceae,g__Vibrio,s__Vibrio sp007124475
2,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,eea831872487c41dfc2fb613f493ba35,31,DNA,0,1000,5828,42,1,GCF_017948435.1 Pseudomonas protegens strain=M...,...,GCF_017948435.1,"((superkingdom, d__Bacteria), (phylum, p__Prot...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Pseudomonadales,f__Pseudomonadaceae,g__Pseudomonas_E,s__Pseudomonas_E putida_H
3,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,2c125d78f91b38497285d44e1b1d5405,31,DNA,0,1000,3306,42,1,"GCA_017995835.1 Amaricoccus sp., ASM1799583v1",...,GCA_017995835.1,"((superkingdom, d__Bacteria), (phylum, p__Prot...",d__Bacteria;p__Proteobacteria;c__Alphaproteoba...,d__Bacteria,p__Proteobacteria,c__Alphaproteobacteria,o__Rhodobacterales,f__Rhodobacteraceae,g__Amaricoccus,s__Amaricoccus sp017995835
4,/group/ctbrowngrp/sourmash-db/gtdb-rs207/gtdb-...,dfc179236087a1222dbe18a61483fa61,31,DNA,0,1000,6960,42,1,GCF_001981135.1 Burkholderia pseudomallei stra...,...,GCF_001981135.1,"((superkingdom, d__Bacteria), (phylum, p__Prot...",d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria,p__Proteobacteria,c__Gammaproteobacteria,o__Burkholderiales,f__Burkholderiaceae,g__Burkholderia,s__Burkholderia mallei


In [6]:
bactInf = sigInf[sigInf["superkingdom"] == "d__Bacteria"]
archInf = sigInf[sigInf["superkingdom"] == "d__Archaea"]

In [7]:
# find taxonomic groups with most genomes, pick first 7 bacteria, 3 archaea
most_common_bact = bactInf.groupby("species")['ident'].count().sort_values(ascending=False)[:7].reset_index()
most_common_bact

,species,ident
0,s__Escherichia coli,26859
1,s__Staphylococcus aureus,13059
2,s__Salmonella enterica,12285
3,s__Klebsiella pneumoniae,11294
4,s__Streptococcus pneumoniae,8452
5,s__Mycobacterium tuberculosis,6836
6,s__Pseudomonas aeruginosa,5623


In [8]:
most_common_arch = archInf.groupby("species")['ident'].count().sort_values(ascending=False)[:3].reset_index()
most_common_arch

,species,ident
0,s__Methanosarcina mazei,73
1,s__Sulfolobus acidocaldarius,56
2,s__Bog-38 sp003162175,29


In [9]:
most_common_species_list =  most_common_bact['species'].tolist() + most_common_arch['species'].tolist()
most_common_species_list

['s__Escherichia coli',
 's__Staphylococcus aureus',
 's__Salmonella enterica',
 's__Klebsiella pneumoniae',
 's__Streptococcus pneumoniae',
 's__Mycobacterium tuberculosis',
 's__Pseudomonas aeruginosa',
 's__Methanosarcina mazei',
 's__Sulfolobus acidocaldarius',
 's__Bog-38 sp003162175']

In [10]:
# load representative genome info
reps = 'gtdb-rs207.taxonomy.reps.csv'
repD = pd.read_csv(reps)
rep_idents = repD['ident'].tolist()
len(rep_idents)

65740

In [11]:
# subset to just reps and just NOT reps
repInf = sigInf[sigInf['ident'].isin(rep_idents)]
print(repInf.shape)
# subset to just reps and just NOT reps
norepInf = sigInf[~sigInf['ident'].isin(rep_idents)]
print(norepInf.shape)

(65740, 23)
(251802, 23)


In [12]:
rep_genomes = repInf[repInf['species'].isin(most_common_species_list)]
rep_genomes.shape

(10, 23)

In [13]:
# list the representative genomes we're using
rep_genomes['name'].tolist()

['GCF_000742135.1 Klebsiella pneumoniae strain=ATCC 13883, ASM74213v1',
 'GCF_000195955.2 Mycobacterium tuberculosis H37Rv strain=H37Rv, ASM19595v2',
 'GCF_001457615.1 Pseudomonas aeruginosa strain=NCTC10332, NCTC10332',
 'GCF_001457635.1 Streptococcus pneumoniae strain=NCTC7465, NCTC7465',
 'GCF_001027105.1 Staphylococcus aureus subsp. aureus DSM 20231 strain=DSM 20231, ASM102710v1',
 'GCA_003162175.1 Euryarchaeota archaeon, 20100900_E2D',
 'GCF_003697165.2 Escherichia coli DSM 30083 = JCM 1649 = ATCC 11775 strain=ATCC 11775, ASM369716v2',
 'GCF_000970205.1 Methanosarcina mazei S-6 strain=S-6, ASM97020v1',
 'GCF_000006945.2 Salmonella enterica subsp. enterica serovar Typhimurium str. LT2, ASM694v2',
 'GCF_000012285.1 Sulfolobus acidocaldarius DSM 639 strain=DSM 639, ASM1228v1']

In [14]:
# get idents for representative genomes 
comparison_idents = rep_genomes['ident'].tolist()
comparison_idents

['GCF_000742135.1',
 'GCF_000195955.2',
 'GCF_001457615.1',
 'GCF_001457635.1',
 'GCF_001027105.1',
 'GCA_003162175.1',
 'GCF_003697165.2',
 'GCF_000970205.1',
 'GCF_000006945.2',
 'GCF_000012285.1']

### Now we also need: genomes in same genus, family, class, phylum for each

for each identifier, walk up the taxonomic ranks and pick 3 genomes that share taxonomy at that rank and not below (e.g. genome from different species in same genus)

In [15]:
compare_idents = comparison_idents
comparisons = []
num_missed=0
n_to_select = 3
for ident in comparison_idents:
    ident_row = sigInf[sigInf['ident']== ident]
    taxlist = tax_utils.ascending_taxlist(include_strain=False)
    prev_tax_at_rank = ""
    prev_rank = "species"
    for rank in taxlist:
        tax_at_rank = list(ident_row[rank])[0]
        #print(tax_at_rank)
        this_info = norepInf[(norepInf[rank] == tax_at_rank)  & (norepInf[prev_rank] != prev_tax_at_rank)]
        if not this_info.empty:
            for n in range(0, n_to_select):
                new_ident = this_info['ident'].tolist()[n]
                if not new_ident:
                    num_missed +=(n_to_select - n)
                    break
                this_comparison = (ident, new_ident)
                comparisons.append(this_comparison)
            prev_tax_at_rank = tax_at_rank
            prev_rank = rank
            #compare_idents.append(new_ident)
        else:
            num_missed+=n_to_select

print(f"num_comparisons:{len(comparisons)}")
print(f"num_missed: {num_missed}")

num_comparisons:192
num_missed: 18


In [16]:
comp = pd.DataFrame.from_records(comparisons, columns=["identA", "identB"])
comp['comparison_reason'] = 'evolpath_for_common_species'
comp

,identA,identB,comparison_reason
0,GCF_000742135.1,GCF_015627065.1,evolpath_for_common_species
1,GCF_000742135.1,GCA_902508775.1,evolpath_for_common_species
2,GCF_000742135.1,GCF_900502275.1,evolpath_for_common_species
3,GCF_000742135.1,GCF_018439335.1,evolpath_for_common_species
4,GCF_000742135.1,GCF_015721435.1,evolpath_for_common_species
...,...,...,...
187,GCF_000012285.1,GCA_001917505.1,evolpath_for_common_species
188,GCF_000012285.1,GCA_011366955.1,evolpath_for_common_species
189,GCF_000012285.1,GCA_002505615.1,evolpath_for_common_species
190,GCF_000012285.1,GCF_017612575.1,evolpath_for_common_species


In [17]:
comp.to_csv("gtdb-rs207.common-sp10-evolpaths.csv", index=False)